In [1]:
!conda install -y -c conda-forge faiss-gpu py-xgboost
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu
    - py-xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



conda-4.14.0         | 1010 KB   | ##################################### | 100% 
toolz-0.12.1         | 51 KB     | ##################################### | 100% 
Preparing tran

In [2]:
# !conda install -y -c conda-forge py-xgboost

In [3]:
# !conda install -y -c conda-forge py-xgboost=*=cuda*

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import faiss
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv("../llamatales/story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [6]:
hidden_states_by_layer = {}
# NUM_PROMPTS = 10
NUM_PROMPTS = 2

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../llamatales/hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
            # print(curr_hidden_states.shape)
            
            #By layer
            for layer in range(1):
                curr_layer_hidden_states = curr_hidden_states[layer][0].astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
                # print(curr_layer_hidden_states.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(curr_layer_hidden_states)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [curr_layer_hidden_states]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:34<00:00, 28.68it/s]


In [7]:
layer_hs_array = hidden_states_by_layer["layer_0"]
print(layer_hs_array[0].shape)
print(layer_hs_array[1].shape)

(270, 512)
(349, 512)


In [8]:
min_story_len = min(df["len_new_story"])
min_story_len

22

In [13]:
#Context-level analysis
# min_story_len = min(df["len_new_story"])
max_story_len = max(df["len_new_story"])
for context_level in range(1, max_story_len):
    curr_context_level_hs = []
    for i in range(len(layer_hs_array)):
        # print(np.array(layer_hs_array[i][:context_level]).shape) #see if taking context level more than available causes error?
        layer_hs_upto_context = layer_hs_array[i][:context_level]
        if(layer_hs_upto_context.shape[0] < context_level):
            # print(i) #see which samples have less tokens than current context level
            continue
        else:
            curr_context_level_hs.append(np.array(layer_hs_upto_context).flatten())
    curr_context_level_hs = np.array(curr_context_level_hs).astype('float32')
    # print([i for i in range(len(curr_context_level_hs)) if curr_context_level_hs[i].shape[0] < i]) #see if any residuals escape screening
    # print(curr_context_level_hs.shape) #print shape without samples less than context level
    
    
    classifier = self.MODEL_CLASSIFIER(**ht_best_params, seed = 42, objective = 'multi:softmax', eval_metric = "merror")
    classifier.fit(self.X_train, self.train_y)
    preds = classifier.predict(self.X_test)
    if(curr_context_level_hs.shape[0] <= NUM_PROMPTS * 1000 // 2):
        break

(2000, 512)
(2000, 1024)
(2000, 1536)
(2000, 2048)
(2000, 2560)
(2000, 3072)
(2000, 3584)
(2000, 4096)
(2000, 4608)
(2000, 5120)
(2000, 5632)
(2000, 6144)
(2000, 6656)
(2000, 7168)
(2000, 7680)
(2000, 8192)
(2000, 8704)
(2000, 9216)
(2000, 9728)
(2000, 10240)
(2000, 10752)
(2000, 11264)
(2000, 11776)
(2000, 12288)
(2000, 12800)
(2000, 13312)
(2000, 13824)
(2000, 14336)
(2000, 14848)
(2000, 15360)
(2000, 15872)
(2000, 16384)
(2000, 16896)
(2000, 17408)
(2000, 17920)
(1999, 18432)
(1999, 18944)
(1999, 19456)
(1999, 19968)
(1999, 20480)
(1999, 20992)
(1999, 21504)
(1999, 22016)
(1999, 22528)
(1999, 23040)
(1999, 23552)
(1999, 24064)
(1999, 24576)
(1999, 25088)
(1999, 25600)
(1999, 26112)
(1999, 26624)
(1999, 27136)
(1998, 27648)
(1998, 28160)
(1998, 28672)
(1997, 29184)
(1997, 29696)
(1997, 30208)
(1997, 30720)
(1995, 31232)
(1993, 31744)
(1992, 32256)
(1990, 32768)
(1990, 33280)
(1986, 33792)
(1985, 34304)
(1982, 34816)
(1982, 35328)
(1982, 35840)
(1976, 36352)
(1975, 36864)
(1970, 37376

In [10]:
# classifier = self.MODEL_CLASSIFIER(**ht_best_params, seed = 42, objective = 'multi:softmax', eval_metric = "merror")
# classifier.fit(self.X_train, self.train_y)
# preds = classifier.predict(self.X_test)